## 1. Importar librerías necesarias

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## 2. Cargar datos desde archivos CSV
- Se importan los archivos de datos crudos desde la carpeta data_raw

In [2]:
ipc = pd.read_csv("../data_raw/ipc.csv", sep=";", encoding="latin-1", decimal=",")
ipv = pd.read_csv("../data_raw/ipv.csv", sep=";", encoding="latin-1", decimal=",")
ipva = pd.read_csv("../data_raw/ipva.csv", sep=";", encoding="latin-1", decimal=",")
ipva.rename(columns={"Ano": "Año"}, inplace=True)
ipva["ipva"] = pd.to_numeric(ipva["ipva"], errors="coerce")
ipva = ipva.groupby("Año", as_index=False)["ipva"].mean()
nuevasviviendas = pd.read_csv("../data_raw/nuevas_viviendas.csv", sep=";", encoding="latin-1", decimal=",")
nuevasviviendas.rename(columns={"Ano": "Año"}, inplace=True)
nuevasviviendas["nuevas_viviendas"] = pd.to_numeric(nuevasviviendas["nuevas_viviendas"], errors="coerce")
salarios = pd.read_csv("../data_raw/salarios.csv", sep=";", encoding="latin-1", decimal=",")
salarios["Salario Medio"] = pd.to_numeric(salarios["Salario Medio"], errors="coerce")

## 3. Procesar IPC (Índice de Precios al Consumo)
- Convertir formato de período a fecha y extraer año
- Calcular promedio anual del IPC

In [3]:
ipc["Fecha"] = pd.to_datetime(
    ipc["Periodo"].str.replace("M", "-") + "-01"
)
ipc["Año"] = ipc["Fecha"].dt.year

ipc_anual = (
    ipc.groupby("Año", as_index=False)["ipc"]
      .mean()
)

## 4. Procesar IPV (Índice de Precios de Vivienda)
- Mapeo de trimestres a meses
- Convertir formato trimestral a fecha
- Calcular promedio anual del IPV

In [4]:
trimestre_a_mes = {
    "T1": "01",
    "T2": "04",
    "T3": "07",
    "T4": "10"
}

ipv["Fecha"] = pd.to_datetime(
    ipv["Periodo"].str[:4] + "-" +
    ipv["Periodo"].str[4:].map(trimestre_a_mes) + "-01"
)

ipv["Año"] = ipv["Fecha"].dt.year

ipv_anual = (
    ipv.groupby("Año", as_index=False)["ipv"]
      .mean()
)

## 5. Fusionar todos los dataframes
- Combinación de datos por año común

In [5]:
df = nuevasviviendas.merge(ipc_anual, on="Año").merge(
	ipv_anual, on="Año").merge(salarios, on="Año").merge(
	ipva, on="Año")

## 6. Normalizar datos (base 2008 = 100)
- Renombrar columna de salario
- Ajustar salario por IPC (deflactar)
- Normalizar cada indicador al año 2008 (índice = 100)
- Convertir año a entero
- Guardar datos limpios en CSV

In [ ]:
df.rename(columns={"Salario Medio": "salario_real"}, inplace=True)
df["salario_real"] = df["salario_real"] / (df["ipc"] / 100)
df["nuevas_viviendas"] = df["nuevas_viviendas"] / (df[df["Año"] == 2008]["nuevas_viviendas"].values[0]) * 100
df["ipc"] = df["ipc"] / (df[df["Año"] == 2008]["ipc"].values[0]) * 100
df["ipv"] = df["ipv"] / (df[df["Año"] == 2008]["ipv"].values[0]) * 100
df["ipva"] = df["ipva"] / (df[df["Año"] == 2008]["ipva"].values[0]) * 100
df["salario_real"] = df["salario_real"] / (df[df["Año"] == 2008]["salario_real"].values[0]) * 100
df['Año'] = df['Año'].astype(int)
df.to_csv("../data_clean/mercado_vivienda_procesado.csv", index=False, sep=";", encoding="utf-8")